In [3]:
# %pip install pandas
# %pip install xlrd
import pandas as pd
import re

In [10]:
# o excel possui comentários até a linha 41 e a linha 43 está vazia
df = pd.read_excel("cmed_final_v3.xlsx")
df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,rec,13,Por subs (1 a 4),Por tipo(1 a 10),Probabilidade
0,22,359,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,1023504310059,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,150,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,Tarja Vermelha (**),1,1,2.5
1,23,76,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501115110025705,1063902720108,NESINA MET,"(12,5 + 1000,0) MG COM REV CT BL AL AL X 60",60,A10N3 - ASSOCIAÇÕES DE INIBIDORES DPP-IV COM B...,Novo,228.00,303.77,Liberado,Tarja Vermelha,1,1,2.5
2,24,470,CARBAMAZEPINA,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,526515003110311,1006800850070,TEGRETOL,200 MG COM CT BL AL PLAS PVC TRANS X 60,60,N3A - ANTIEPILÉPTICOS,Novo,64.54,89.22,C1,Tarja Vermelha sob restrição,1,1,2.5
3,25,496,CLORIDRATO DE SERTRALINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528519120163307,1256802880219,TRELIM,100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60,60,N6A4 - ANTIDEPRESSIVOS SSRI,Similar,290.24,401.24,C1,Tarja Vermelha sob restrição,1,1,2.5
4,26,300,EZETIMIBA,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,543817070024603,1351700300049,ZETERINA,10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60,60,C10A9 - TODOS OUTROS REGULADORES DE COLESTEROL...,Similar,200.62,277.35,Liberado,Tarja Vermelha,1,1,2.5


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        505 non-null    int64  
 1   0                 505 non-null    int64  
 2   1                 505 non-null    object 
 3   2                 505 non-null    object 
 4   3                 505 non-null    object 
 5   4                 505 non-null    int64  
 6   5                 505 non-null    int64  
 7   6                 505 non-null    object 
 8   7                 505 non-null    object 
 9   8                 505 non-null    int64  
 10  9                 505 non-null    object 
 11  10                505 non-null    object 
 12  11                505 non-null    float64
 13  12                505 non-null    float64
 14  rec               505 non-null    object 
 15  13                505 non-null    object 
 16  Por subs (1 a 4)  505 non-null    int64  
 1

In [12]:
# selecionar somente linhas desejadas
df = df.filter(items=[1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 'rec', 'Probabilidade'])

# renomear algumas colunas
df = df.rename(columns={1: 'substância', 2: 'cnpj', 3: 'laboratório', 4: 'ggrem', 6: 'produto', 7: 'apresentacao', 8: 'quantidade', 9: 'classe', 10: 'tipo', 11: 'custo', 12: 'preço', 'rec': 'controle'})
df.head()

,substância,cnpj,laboratório,ggrem,produto,apresentacao,quantidade,classe,tipo,custo,preço,controle,Probabilidade
0,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,150,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,2.5
1,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501115110025705,NESINA MET,"(12,5 + 1000,0) MG COM REV CT BL AL AL X 60",60,A10N3 - ASSOCIAÇÕES DE INIBIDORES DPP-IV COM B...,Novo,228.00,303.77,Liberado,2.5
2,CARBAMAZEPINA,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,526515003110311,TEGRETOL,200 MG COM CT BL AL PLAS PVC TRANS X 60,60,N3A - ANTIEPILÉPTICOS,Novo,64.54,89.22,C1,2.5
3,CLORIDRATO DE SERTRALINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528519120163307,TRELIM,100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60,60,N6A4 - ANTIDEPRESSIVOS SSRI,Similar,290.24,401.24,C1,2.5
4,EZETIMIBA,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,543817070024603,ZETERINA,10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60,60,C10A9 - TODOS OUTROS REGULADORES DE COLESTEROL...,Similar,200.62,277.35,Liberado,2.5


In [33]:
# df.iloc[1:100, 4]

pt1 = r'SUS|COM|CAP|SOL|XPE|CREM|POM|INJ|IM|GEL|EMU|PAST|LOCAO|XAMP|DRG|$'

cont = {'SUS': 0, 'COM': 0, 'CAP': 0, 'SOL': 0, 'XPE': 0, 'CREM': 0, 'POM': 0}
for a in df['apresentacao']:
  # if re.search(pt1, a) and re.search(pt1, a).group() == 'COM':
  #   print(re.split(pt1, a)[0], '=>', re.search(pt1, a).group(), '|', a)
  print(re.split(pt1, a)[0], '=>', re.search(pt1, a).group(), '|', a)

  # if re.search(pt1, a) and re.search(pt1, a).group() in cont:
  #   cont[re.search(pt1, a).group()] += 1
  # else:
  #   print(re.split(pt1, a)[0], '=>', re.search(pt1, a).group(), '|', a)

print(cont)

50 MG/ML PO P/  => SUS | 50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML 
(12,5 + 1000,0) MG  => COM | (12,5 + 1000,0) MG COM REV CT BL AL AL X 60
200 MG  => COM | 200 MG COM CT BL AL PLAS PVC TRANS X 60
100 MG  => COM | 100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60
10 MG  => COM | 10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60
10 MG  => COM | 10 MG COM REV CT BL AL/AL X 60
20 MG  => COM | 20 MG COM REV CT BL AL/AL X 30
12,5 MG  => COM | 12,5 MG COM REV CT BL AL  AL X 30
(25,0 + 15,0) MG  => COM | (25,0 + 15,0) MG COM REV CT BL AL AL X 30
8 MG  => COM | 8 MG COM CT BL AL PLAS TRANS X 30
100 MG  => COM | 100 MG COM REV LIB PROL CT BL AL PLAS INC X 30 
80MG + 10MG  => COM | 80MG + 10MG COM CT BL AL/AL X 30
(2,0 + 5,0) MG   => COM | (2,0 + 5,0) MG  COM REV CAP DURA CT BL AL PLAS PVC/PVDC TRANS X 30
(2,0 + 5,0) MG  => COM | (2,0 + 5,0) MG COM REV CAP DURA CT BL AL PLAS PVC/PVDC TRANS X 30
150 MG  => COM | 150 MG COM REV LIB PROL CT BL AL AL X 30 
(500+125) MG  => COM | (500+125) MG COM REV CT 

In [8]:
df.shape

(505, 13)

In [34]:
pt1 = r'SUS|COM|CAP|SOL|XPE|CREM|POM|INJ|IM|GEL|EMU|PAST|LOCAO|XAMP|DRG|$'

for i in range(df.shape[0]):
  apres = df['apresentacao'][i]
  rg = re.search(pt1, apres).group()
  quant = df['quantidade'][i]
  print(quant, rg)

150 SUS
60 COM
60 COM
60 COM
60 COM
60 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 CAP
30 COM
30 COM
28 COM
28 CAP
28 CAP
28 COM
28 COM
28 COM
28 CAP
28 COM
20 COM
20 COM
14 COM
14 COM
14 CAP
14 COM
12 COM
12 CAP
10 COM
9 COM
5 INJ
5 POM
5 SOL
4 SOL
4 CAP
3 COM
2 IM
0 SUS
150 SUS
90 COM
60 COM
60 COM
60 COM
60 COM
60 COM
30 CAP
30 CAP
30 CAP
30 COM
28 CAP
21 CAP
20 COM
20 COM
16 COM
10 COM
8 COM
7 COM
6 COM
2 SUS
1 COM
0 SOL
90 COM
60 COM
60 COM
60 COM
60 COM
60 COM
60 COM
60 COM
60 COM
50 COM
50 COM
50 COM
30 COM
30 COM
30 COM
30 COM
30 SOL
30 COM
30 GEL
30 COM
30 COM
30 COM
30 COM
30 CAP
30 COM
30 COM
30 COM
30 COM
30 COM
30 COM
30 CAP
28 COM
28 CAP
28 CAP
28 COM
28 COM
25 CAP
25 COM
25 CAP
25 CAP
20 XPE
20 COM
20 SOL
20 SOL
20 COM
20 CAP
20 COM
15 COM
15 CREM
15 SOL
14 COM
12 SUS
10 COM
10 COM
10 COM
10 COM
10 CAP
10 COM
10 CAP
8 CAP
5 SOL
5 COM
5 IM
5 COM
5 SOL
3 COM
3 SOL
3 CREM
2 SOL
1 COM
0 SOL
0 SUS
0 SUS
0 SOL

Criando colunas "DOSAGEM", "APRESENTACAO", "CONTROLE", "QUANTIDADE", "CLASSE"

In [35]:
# ps = ['MG\s\/\sML', 'ML/ML', 'MG/ML', 'MG/G', 'GV/ML', 'MEQ/L', 'G/L', 'G/G', 'MG', 'MCG', 'G/ML', 'TRU', 'MU/ML', 'UI/ML', 'U/G', 'U/U', '\sUI\s', '\sU\s', '\sML\s', 'SOL\sINJ', '$']
pt1 = r'MG\s\/\sML|ML/ML|MG/ML|MG/G|GV/ML|MEQ/L|G/L|G/G|MG|MCG|G/ML|TRU|MU/ML|UI/ML|U/G|U/U|\sUI\s|\sU\s|\sML\s|SOL\sINJ|$'
pt2 = r'X\s'
pt3 = r'POM|CREM|GEL|MOLE|EMU|PAST|LOCAO|XAMP'
pt4 = r'SOL|XPE|SUS|INJ|ML|IM'
pt5 = r' - '

# pt1 = r'SUS|COM|CAP|SOL|XPE|CREM|POM|INJ|IM|GEL|EMU|PAST|LOCAO|XAMP|DRG|$'

dosagem = []
apresentacao = []
controle = []
classe = []
quantidade = []


for a in df['apresentacao']:
  # for x in f
  m1 = re.search(pt1, a)
  d = re.split(pt1, a)[0] + m1.group() if m1 else ''

  m2 = re.split(pt2, a)
  q = m2[-1] if m2 else ''

  m3 = re.search(pt3, a)
  m4 = re.search(pt4, a)
  if m3: t = 'Semissólido'
  elif m4: t = 'Líquido'
  else: t = 'Sólido'

  dosagem.append(d)
  quantidade.append(q)
  apresentacao.append(t)

  # print(a, '=> DOSAGEM:', d, '| QUANTIDADE:', q, '| APRESENTACAO:', t)
  

for a in df['classe']:
  co = re.split(pt5, a)[0][:2]
  co = co if co in ['A2', 'A3', 'B1', 'B2', 'C1', 'C2', 'AM'] else 'Não controlado'
  controle.append(co)
  
  cl = re.split(pt5, a)[1]
  classe.append(cl)
  # print(a, '=> CONTROLE:', co, '| CLASSE:', cl)
  
df['DOSAGEM T'] = dosagem
df['APRESENTACAO T'] = apresentacao
df['CONTROLE T'] = controle # rec
df['QUANTIDADE T'] = quantidade
df['CLASSE T'] = classe

In [174]:
df.loc[df['TIPO'] == '    -     ', 'TIPO'] = 'Outros' #10

In [36]:
df.head(10)

,substância,cnpj,laboratório,ggrem,produto,apresentacao,quantidade,classe,tipo,custo,preço,controle,Probabilidade,DOSAGEM T,APRESENTACAO T,CONTROLE T,QUANTIDADE T,CLASSE T
0,AMPICILINA,57.507.378/0003-65,EMS S/A,507701907133117,AMPICILINA,50 MG/ML PO P/ SUS OR CT FR VD AMB X 150 ML,150,J1C1 - PENICILINAS ORAIS DE AMPLO ESPECTRO,Genérico,53.61,74.11,AM,2.5,50 MG/ML,Líquido,Não controlado,150 ML,PENICILINAS ORAIS DE AMPLO ESPECTRO
1,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501115110025705,NESINA MET,"(12,5 + 1000,0) MG COM REV CT BL AL AL X 60",60,A10N3 - ASSOCIAÇÕES DE INIBIDORES DPP-IV COM B...,Novo,228.00,303.77,Liberado,2.5,"(12,5 + 1000,0) MG",Sólido,Não controlado,60,ASSOCIAÇÕES DE INIBIDORES DPP-IV COM BIGUANIDAS
2,CARBAMAZEPINA,56.994.502/0001-30,NOVARTIS BIOCIENCIAS S.A,526515003110311,TEGRETOL,200 MG COM CT BL AL PLAS PVC TRANS X 60,60,N3A - ANTIEPILÉPTICOS,Novo,64.54,89.22,C1,2.5,200 MG,Sólido,Não controlado,60,ANTIEPILÉPTICOS
3,CLORIDRATO DE SERTRALINA,73.856.593/0001-66,PRATI DONADUZZI & CIA LTDA,528519120163307,TRELIM,100 MG COM REV CT BL AL PLAS PVC/PVDC TRANS X 60,60,N6A4 - ANTIDEPRESSIVOS SSRI,Similar,290.24,401.24,C1,2.5,100 MG,Sólido,Não controlado,60,ANTIDEPRESSIVOS SSRI
4,EZETIMIBA,48.344.725/0007-19,ALTHAIA S.A INDÚSTRIA FARMACÊUTICA,543817070024603,ZETERINA,10 MG COM CT BL AL PLAS PVDC/PVC TRANS X 60,60,C10A9 - TODOS OUTROS REGULADORES DE COLESTEROL...,Similar,200.62,277.35,Liberado,2.5,10 MG,Sólido,C1,60,TODOS OUTROS REGULADORES DE COLESTEROL E TRIGL...
5,ATORVASTATINA CÁLCICA,46.070.868/0036-99,LABORATÓRIOS PFIZER LTDA,552820050068217,CITALOR,10 MG COM REV CT BL AL/AL X 60,60,"C10A1 - ESTATINAS, INIBIDORES DA REDUTASE HMG-CoA",Novo,199.18,275.35,Liberado,2.5,10 MG,Sólido,C1,60,"ESTATINAS, INIBIDORES DA REDUTASE HMG-CoA"
6,ROSUVASTATINA CÁLCICA,60.659.463/0029-92,ACHÉ LABORATÓRIOS FARMACÊUTICOS S.A,500512070043204,TREZOR,20 MG COM REV CT BL AL/AL X 30,30,"C10A1 - ESTATINAS, INIBIDORES DA REDUTASE HMG-CoA",Similar,285.95,395.31,Liberado,2.5,20 MG,Sólido,C1,30,"ESTATINAS, INIBIDORES DA REDUTASE HMG-CoA"
7,BENZOATO DE ALOGLIPTINA,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501114030022102,NESINA,"12,5 MG COM REV CT BL AL AL X 30",30,A10N1 - ANTIDIABÉTICOS INIBIDORES DPP-IV PUROS,Novo,115.54,153.93,Liberado,2.5,"12,5 MG",Sólido,Não controlado,30,ANTIDIABÉTICOS INIBIDORES DPP-IV PUROS
8,BENZOATO DE ALOGLIPTINA;CLORIDRATO DE PIOGLITA...,60.397.775/0001-74,TAKEDA PHARMA LTDA.,501116070027205,NESINA PIO,"(25,0 + 15,0) MG COM REV CT BL AL AL X 30",30,A10N9 - OUTRAS ASSOCIAÇÕES DE ANTIDIABÉTICOS C...,Novo,226.40,301.63,Liberado,2.5,"(25,0 + 15,0) MG",Sólido,Não controlado,30,OUTRAS ASSOCIAÇÕES DE ANTIDIABÉTICOS COM INIBI...
9,CANDESARTANA CILEXETILA,60.318.797/0001-00,ASTRAZENECA DO BRASIL LTDA,502300304119219,ATACAND,8 MG COM CT BL AL PLAS TRANS X 30,30,C9C - ANTAGONISTAS DA ANGIOTENSINA II PUROS,Novo,143.43,198.28,Liberado,2.5,8 MG,Sólido,Não controlado,30,ANTAGONISTAS DA ANGIOTENSINA II PUROS


0                                           AMPICILINA
1     BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA
2                                        CARBAMAZEPINA
3                             CLORIDRATO DE SERTRALINA
4                                            EZETIMIBA
                            ...                       
95                            BROMIDRATO DE CITALOPRAM
96                       HEMITARTARATO DE RIVASTIGMINA
97                                          VALSARTANA
98                             OXALATO DE ESCITALOPRAM
99                 ESOMEPRAZOL MAGNÉSICO TRI-HIDRATADO
Name: substância, Length: 100, dtype: object

In [23]:
[e for e in df['substância'][:100]]

['AMPICILINA',
 'BENZOATO DE ALOGLIPTINA;CLORIDRATO DE METFORMINA',
 'CARBAMAZEPINA',
 'CLORIDRATO DE SERTRALINA',
 'EZETIMIBA',
 'ATORVASTATINA CÁLCICA',
 'ROSUVASTATINA CÁLCICA',
 'BENZOATO DE ALOGLIPTINA',
 'BENZOATO DE ALOGLIPTINA;CLORIDRATO DE PIOGLITAZONA',
 'CANDESARTANA CILEXETILA',
 'SUCCINATO DE METOPROLOL',
 'BESILATO DE ANLODIPINO;TELMISARTANA',
 'MESILATO DE DOXAZOSINA;FINASTERIDA',
 'MESILATO DE DOXAZOSINA;FINASTERIDA',
 'CLORIDRATO DE BUPROPIONA',
 'CLAVULANATO DE POTÁSSIO;AMOXICILINA TRI-HIDRATADA',
 'CARBIDOPA (PORT. 344/98 LISTA C 1);LEVODOPA',
 'CILOSTAZOL',
 'BESILATO DE ANLODIPINO;HEMIFUMARATO DE BISOPROLOL',
 'EZETIMIBA',
 'ACICLOVIR',
 'CLORIDRATO DE VENLAFAXINA',
 'HEMITARTARATO DE ZOLPIDEM',
 'ACICLOVIR',
 'ESOMEPRAZOL MAGNÉSICO',
 'CLORIDRATO DE VENLAFAXINA',
 'LANSOPRAZOL;AMOXICILINA TRI-HIDRATADA;CLARITROMICINA',
 'HEMIFUMARATO DE BISOPROLOL',
 'ESOMEPRAZOL MAGNÉSICO',
 'VALSARTANA',
 'BROMIDRATO DE GALANTAMINA',
 'CLARITROMICINA',
 'CANDESARTANA CILEXETILA'

Filtro dos medicamentos

In [177]:
df_filtro = df.loc[
  (df.PREÇO < 300) &
  (df.PRODUTO.isin(['BUSCOPAN', 'DEPOSTERON', 'ACCUVIT', 'ACEBROFILINA']))
]
df_filtro

,SUBSTÂNCIA,CNPJ,LABORATÓRIO,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO,PREÇO,DOSAGEM,APRESENTACAO,CONTROLE,QUANTIDADE,CLASSE
0,BUTILBROMETO DE ESCOPOLAMINA,61.082.426/0002-07,COSMED INDUSTRIA DE COSMETICOS E MEDICAMENTOS ...,BUSCOPAN,20 MG SOL INJ CT 5 AMP VD TRANS X 1 ML,A3A - ANTIESPASMÓDICOS E ANTICOLINÉRGICOS PUROS,Novo,11.13,20 MG,Líquido,A3,1 ML,ANTIESPASMÓDICOS E ANTICOLINÉRGICOS PUROS
3,CIPIONATO DE TESTOSTERONA,00.923.140/0001-31,EMS SIGMA PHARMA LTDA,DEPOSTERON,100 MG/ML SOL INJ CX 3 AMP VD AMB X 2 ML,"G3B - ANDRÓGENOS EXCLUINDO G3E, G3F",Similar,200.12,100 MG/ML,Líquido,Não controlado,2 ML,"ANDRÓGENOS EXCLUINDO G3E, G3F"
7,ACEBROFILINA,17.159.229/0001-76,LABORATÓRIO TEUTO BRASILEIRO S/A,ACEBROFILINA,10 MG/ML XPE CT FR PET AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,29.07,10 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES
8,ACEBROFILINA,17.159.229/0001-76,LABORATÓRIO TEUTO BRASILEIRO S/A,ACEBROFILINA,5MG/ML XPE CT FR PET AMB X 120ML + COP,R5C - EXPECTORANTES,Genérico,20.84,5MG/ML,Líquido,Não controlado,120ML + COP,EXPECTORANTES
58,ACEBROFILINA,61.190.096/0001-92,EUROFARMA LABORATÓRIOS S.A.,ACEBROFILINA,5 MG/ML XPE CT FR VD AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,20.84,5 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES
59,ACEBROFILINA,61.190.096/0001-92,EUROFARMA LABORATÓRIOS S.A.,ACEBROFILINA,10 MG/ML XPE CT FR VD AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,29.38,10 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES
63,ACEBROFILINA,57.507.378/0003-65,EMS S/A,ACEBROFILINA,5 MG/ML XPE CT FR VD AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,20.80,5 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES
64,ACEBROFILINA,57.507.378/0003-65,EMS S/A,ACEBROFILINA,10 MG/ML XPE CT FR VD AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,29.38,10 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES
65,ACEBROFILINA,57.507.378/0003-65,EMS S/A,ACEBROFILINA,5 MG/ML XPE CT FR PLAS AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,20.80,5 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES
66,ACEBROFILINA,57.507.378/0003-65,EMS S/A,ACEBROFILINA,10 MG/ML XPE CT FR PLAS AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,29.38,10 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES


Uma substância por laboratório ou somente uma substância

In [201]:
# df_filtro.groupby('CNPJ').first().groupby('SUBSTÂNCIA').first()
df_filtro.groupby('CNPJ').first()

,SUBSTÂNCIA,LABORATÓRIO,PRODUTO,APRESENTAÇÃO,CLASSE TERAPÊUTICA,TIPO,PREÇO,DOSAGEM,APRESENTACAO,CONTROLE,QUANTIDADE,CLASSE
CNPJ,,,,,,,,,,,,
00.923.140/0001-31,CIPIONATO DE TESTOSTERONA,EMS SIGMA PHARMA LTDA,DEPOSTERON,100 MG/ML SOL INJ CX 3 AMP VD AMB X 2 ML,"G3B - ANDRÓGENOS EXCLUINDO G3E, G3F",Similar,200.12,100 MG/ML,Líquido,Não controlado,2 ML,"ANDRÓGENOS EXCLUINDO G3E, G3F"
02.814.497/0001-07,ACEBROFILINA,CIMED INDUSTRIA S.A,ACEBROFILINA,10 MG/ML XPE CT FR PLAS AMB X 120 ML + CP MED,R5C - EXPECTORANTES,Genérico,27.59,10 MG/ML,Líquido,Não controlado,120 ML + CP MED,EXPECTORANTES
03.485.572/0001-04,ACEBROFILINA,GEOLAB INDÚSTRIA FARMACÊUTICA S/A,ACEBROFILINA,10MG/ML XPE CT FR VD AMB X 120ML + COP,R5C - EXPECTORANTES,Genérico,32.68,10MG/ML,Líquido,Não controlado,120ML + COP,EXPECTORANTES
05.044.984/0001-26,ACEBROFILINA,LEGRAND PHARMA INDÚSTRIA FARMACÊUTICA LTDA,ACEBROFILINA,5 MG/ML XPE CT FR VD AMB X 120 ML + CP MED,R5C - EXPECTORANTES,Genérico,22.12,5 MG/ML,Líquido,Não controlado,120 ML + CP MED,EXPECTORANTES
05.161.069/0001-10,ACEBROFILINA,BRAINFARMA INDÚSTRIA QUÍMICA E FARMACÊUTICA S.A,ACEBROFILINA,5 MG/ML XPE CT FR VD AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,20.77,5 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES
10.588.595/0010-92,ACEBROFILINA,SANOFI MEDLEY FARMACÊUTICA LTDA.,ACEBROFILINA,5 MG/ML XPE CT FR VD AMB X 120 ML + DOSADOR,R5C - EXPECTORANTES,Genérico,20.79,5 MG/ML,Líquido,Não controlado,120 ML + DOSADOR,EXPECTORANTES
17.115.437/0001-73,ACEBROFILINA,LABORATÓRIO GLOBO SA,ACEBROFILINA,5 MG/ML XPE CT FR PLAS AMB X 120 ML + CP MED,R5C - EXPECTORANTES,Genérico,16.94,5 MG/ML,Líquido,Não controlado,120 ML + CP MED,EXPECTORANTES
17.159.229/0001-76,ACEBROFILINA,LABORATÓRIO TEUTO BRASILEIRO S/A,ACEBROFILINA,10 MG/ML XPE CT FR PET AMB X 120 ML + COP,R5C - EXPECTORANTES,Genérico,29.07,10 MG/ML,Líquido,Não controlado,120 ML + COP,EXPECTORANTES
17.875.154/0001-20,ACEBROFILINA,MEDQUIMICA INDUSTRIA FARMACEUTICA LTDA.,ACEBROFILINA,5 MG/ML XPE CT FR PLAS AMB X 120 ML + CP MED,R5C - EXPECTORANTES,Genérico,20.90,5 MG/ML,Líquido,Não controlado,120 ML + CP MED,EXPECTORANTES


Laboratórios

In [203]:
# pd.concat([df['CNPJ'], df['LABORATÓRIO']]).unique()
df_lab = df_filtro.filter(items=['CNPJ', 'LABORATÓRIO']).drop_duplicates(subset=['CNPJ']).reset_index(drop=True)

nome = []
for e in df_lab['LABORATÓRIO']:
  s = e
  s = s.replace('LABORATÓRIOS', '').strip()
  s = s.replace('LABORATÓRIO', '').strip()
  s = s.replace('LABORATORIO', '').strip()
  s = s.replace('FARMACÊUTICO', '').strip()
  s = s.replace('FARMACÊUTICA', '').strip()
  s = s.replace('INDUSTRIAL', '').strip()
  s = s.replace('INDÚSTRIA', '').strip()
  s = s.replace('INDUSTRIA', '').strip()
  s = s.replace('PRODUTOS', '').strip()
  s = s.replace('PRODUTOS', '').strip()
  s = s[:3] + re.split(r'\.|\s', s[3:])[0]
  s = s.replace('  ', ' ').strip()
  # print(s, '=>', e)
  nome.append(s)
  
df_lab['NOME'] = nome

df_lab.head(10)

,CNPJ,LABORATÓRIO,NOME
0,61.082.426/0002-07,COSMED INDUSTRIA DE COSMETICOS E MEDICAMENTOS ...,COSMED
1,00.923.140/0001-31,EMS SIGMA PHARMA LTDA,EMS
2,17.159.229/0001-76,LABORATÓRIO TEUTO BRASILEIRO S/A,TEUTO
3,61.190.096/0001-92,EUROFARMA LABORATÓRIOS S.A.,EUROFARMA
4,57.507.378/0003-65,EMS S/A,EMS
5,17.115.437/0001-73,LABORATÓRIO GLOBO SA,GLOBO
6,60.659.463/0029-92,ACHÉ LABORATÓRIOS FARMACÊUTICOS S.A,ACHÉ
7,45.992.062/0001-65,GERMED FARMACEUTICA LTDA,GERMED
8,17.875.154/0001-20,MEDQUIMICA INDUSTRIA FARMACEUTICA LTDA.,MEDQUIMICA
9,53.162.095/0001-06,BIOSINTÉTICA FARMACÊUTICA LTDA,BIOSINTÉTICA


In [ ]:
# import psycopg2
# import numpy as np
# import pandas as pd
  
# def execute_values(conn, df, table): 
#   tuples = [tuple(x) for x in df.to_numpy()] 
#   cols = ','.join(list(df.columns))

#   # SQL query to execute 
#   query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols) 
#   cursor = conn.cursor()
#   try: 
#       psycopg2.extras.execute_values(cursor, query, tuples) 
#       conn.commit() 
#   except (Exception, psycopg2.DatabaseError) as error: 
#       print("Error: %s" % error) 
#       conn.rollback() 
#       cursor.close() 
#       return 1
#   print("the dataframe is inserted") 
#   cursor.close() 

# conn = psycopg2.connect( 
#   database="oscar", user='oscar', password='oscar', host='127.0.0.1', port='5432'
# ) 

# df = pd.read_csv('fossilfuels.csv') 

# execute_values(conn, df, 'fossil_fuels_c02') 